In [156]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, ShuffleSplit
from sklearn.metrics import make_scorer, r2_score
import pandas as pd

In [139]:
data = np.load("climbing_dataset.npz", allow_pickle=True)
holds = data["X"]
y = data["y"]
names = data["names"]
global_features = data["global_features"]
print(global_features.shape, holds.shape, y.shape)

(3296, 2) (3296, 2, 498) (3296,)


In [140]:
X_holds = holds.reshape(holds.shape[0], -1)
X = np.concatenate([X_holds, global_features], axis=1)

print(X.shape, y.shape)

(3296, 998) (3296,)


In [ ]:
bins = np.arange(10, 31, 1)
y_binned = np.digitize(y, bins, right=False)
X_train, X_test = None, None
y_train, y_test = None, None
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in split.split(X, y_binned):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

In [ ]:
import xgboost as xgb 
model = xgb.XGBRegressor(   objective='reg:squarederror',   
                            learning_rate=0.005, 
                            max_depth=0, 
                            random_state=42,
                            eval_metric = 'rmse',
                            subsample = 0.1,
                            colsample_bytree =  0.1, 
                            n_estimators=20000,
                            gamma = 10)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder


#regressor = RandomForestRegressor(n_estimators=1000, random_state=0, oob_score=True)


regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [153]:
param_grid = {
    #'num_leaves': [31, 63, 127],
    'learning_rate': [0.1, 0.05, 0.01, .005],
    'n_estimators': [200, 500, 1000],
    'max_depth': [-1, 2,3, 5],
    'subsample': [0.2,0.4,0.6],
    'colsample_bytree': [0.2,0.4,0.6,]
}

scorer = make_scorer(mean_squared_error)  # or use neg_mean_squared_error, etc.
cv_splitter = ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
# -----------------------------------
# Grid search
# -----------------------------------
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring=scorer,
    cv=cv_splitter,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# -----------------------------------
# Results
# -----------------------------------
print("Best params:", grid_search.best_params_)
print("Best CV R²:", grid_search.best_score_)

# Evaluate on test set
best_model = grid_search.best_estimator_
test_r2 = r2_score(y_test, best_model.predict(X_test))
print("Test R²:", test_r2)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


c:\Users\shado\Documents\Development\TB2 Project\TB2-Classifier\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
540 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\shado\Documents\Development\TB2 Project\TB2-Classifier\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\shado\Documents\Development\TB2 Project\TB2-Classifier\.venv\Lib\site-packages\xgboost\core.py", line 774, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\shado\Docume

Best params: {'colsample_bytree': 0.2, 'learning_rate': 0.005, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.2}
Best CV R²: 15.983644305398943
Test R²: 0.26753922219937964


In [163]:
results_df = pd.DataFrame(grid_search.cv_results_)
results_df = results_df.dropna()

results_df_sorted = results_df.sort_values(by='mean_test_score', ascending=True)
# Show all columns (optional)
pd.set_option('display.max_columns', None)

# Display the first few rows
results_df_sorted.head()


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
215,4.973810,0.102138,0.024201,0.008034,0.4,0.05,5,1000,0.6,"{'colsample_bytree': 0.4, 'learning_rate': 0.0...",2.071121,2.521604,2.294859,2.356078,2.378465,2.324425,0.146860,324
35,4.945161,0.047972,0.035102,0.004054,0.2,0.10,5,1000,0.6,"{'colsample_bytree': 0.2, 'learning_rate': 0.1...",2.063094,2.570905,2.224817,2.380786,2.478250,2.343570,0.181093,323
214,5.006477,0.042703,0.018600,0.000800,0.4,0.05,5,1000,0.4,"{'colsample_bytree': 0.4, 'learning_rate': 0.0...",2.182863,2.561723,2.402975,2.301096,2.427372,2.375206,0.127098,322
359,5.257670,0.132196,0.027400,0.008777,0.6,0.05,5,1000,0.6,"{'colsample_bytree': 0.6, 'learning_rate': 0.0...",2.150157,2.567801,2.317343,2.406710,2.438236,2.376050,0.138600,321
70,4.832805,0.094095,0.027302,0.008329,0.2,0.05,5,1000,0.4,"{'colsample_bytree': 0.2, 'learning_rate': 0.0...",2.116235,2.552281,2.372718,2.430647,2.427630,2.379902,0.144319,320


In [222]:


train_rsme = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'Train RMSE: {train_rsme:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'R²: {r2:.3f}')

Train RMSE: 0.444
RMSE: 1.270
R²: 0.927
